In [9]:
import os
import json
import numpy as np

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten, Dense, Activation
from tensorflow.keras.layers import InputLayer

# os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

## 1. Model Build & Summary

### Sequential 이용

In [2]:
model = Sequential()
model.add(Flatten())
model.add(Dense(units=10, activation='relu'))  # activation 지정하는 1번째 방법
model.add(Dense(units=2))
model.add(Activation('softmax'))  # activation 지정하는 2번째 방법

print(model.summary())

ValueError: ignored

input이 정의되지 않은 상태이기 때문에 오류가 발생했다.  
summary()를 사용하기 전에 build를 해서, input 데이터의 형태를 알려줘야 한다

In [3]:
tf.keras.backend.clear_session()   # 세션을 초기화 -> layer들에 붙인 이름을 초기화 (다시 dense, dense_1,...부터 시작) 
                      # batch, width, height, channel
model.build(input_shape=(None, 28, 28, 1)) # batch의 크기는 달라질 수 있기 때문에 None으로 지정
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 10)                7850      
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 22        
_________________________________________________________________
activation (Activation)      (None, 2)                 0         
Total params: 7,872
Trainable params: 7,872
Non-trainable params: 0
_________________________________________________________________


28\*28 = 784  
dense의 parameter 개수 7850 = 784\*10 + 10  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(Output shape)&nbsp;&nbsp;(bias)  
dense의 parameter 개수 22 = 10*2 + 2

### Model Subclass 이용

In [4]:
class TestModel(Model):
    def __init__(self):
        super(TestModel, self).__init__()

        self.flatten = Flatten()
        self.d1 = Dense(units=10)
        self.d1_act = Activation('relu')
        self.d2 = Dense(units=2)
        self.d2_act = Activation('softmax')

    def call(self, x):
        x = self.flatten(x)
        x = self.d1(x)
        x = self.d1_act(x)
        x = self.d2(x)
        x = self.d2_act(x)
        return x

In [5]:
model = TestModel()
model.build(input_shape=(None, 28, 28, 1))

model.summary()

Model: "test_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            multiple                  0         
_________________________________________________________________
dense (Dense)                multiple                  7850      
_________________________________________________________________
activation (Activation)      multiple                  0         
_________________________________________________________________
dense_1 (Dense)              multiple                  22        
_________________________________________________________________
activation_1 (Activation)    multiple                  0         
Total params: 7,872
Trainable params: 7,872
Non-trainable params: 0
_________________________________________________________________


## 2. InputLayer 이용

In [10]:
model = Sequential()
model.add(InputLayer(input_shape=(28, 28, 1)))  # batch size는 넣지 않는다
model.add(Flatten())
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))

# model.build()  # 따로 build를 하지 않아도 summary()를 구할 수 있다

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 10)                7850      
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 22        
_________________________________________________________________
activation_2 (Activation)    (None, 2)                 0         
Total params: 7,872
Trainable params: 7,872
Non-trainable params: 0
_________________________________________________________________


=> 하지만 InputLayer는 Model Subclassing에는 사용할 수 없다  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;두 방식에서 통일성을 주기 위해서는 둘다 model.build(input_shape=(...))으로 지정하는게 좋다

## 3. Built

In [12]:
model = Sequential()
model.add(Flatten())
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))

print("[before]", model.built)
model.build(input_shape=(None, 28, 28, 3))
print("[after]", model.built)

[before] False
[after] True


## 4. 데이터를 통과시켜서 모델 빌드시키기

In [13]:
model = Sequential()
model.add(Flatten())
model.add(Dense(units=10, activation='relu'))
model.add(Dense(units=2))
model.add(Activation('softmax'))

print("[before]", model.built)
test_img = tf.random.normal(shape=(1, 28, 28, 1))
model(test_img)
print("[after]", model.built)

[before] False
[after] True


=> 데이터를 통과시키면 그것에 맞춰서 자동으로 모델이 build 된다  
=> 학습을 돌리기 전에 분석을 하기 위해서는 model.build() API를 사용하고,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;그렇지 않을 때는 따로 사용할 필요 없다

## 5. CNN에의 적용

In [14]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.layers import Flatten, Dense, Activation

In [18]:
model = Sequential()
model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='valid', name='conv_1'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='conv_1_maxpool'))
model.add(Activation('relu', name='conv_1_act'))

model.add(Conv2D(filters=10, kernel_size=(3, 3), padding='valid', name='conv_2'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, name='conv_2_maxpool'))
model.add(Activation('relu', name='conv_2_act'))

model.add(Flatten())
model.add(Dense(units=32, activation='relu', name='dense_1'))
model.add(Dense(units=10, activation='softmax', name='softmax'))

In [19]:
model.build(input_shape=(None, 28, 28, 1))
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_1 (Conv2D)              (None, 26, 26, 10)        100       
_________________________________________________________________
conv_1_maxpool (MaxPooling2D (None, 13, 13, 10)        0         
_________________________________________________________________
conv_1_act (Activation)      (None, 13, 13, 10)        0         
_________________________________________________________________
conv_2 (Conv2D)              (None, 11, 11, 10)        910       
_________________________________________________________________
conv_2_maxpool (MaxPooling2D (None, 5, 5, 10)          0         
_________________________________________________________________
conv_2_act (Activation)      (None, 5, 5, 10)          0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 250)              